# Network Blending Experiment

## Setup Google Drive Connection

In [1]:
# Check GPU connection
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-46ea0376-9f9e-07d0-1d6d-17380540b14b)


In [2]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
!pip install gdown --upgrade

if os.path.isdir("/content/drive/MyDrive/stylegan3-fun-blend"):
    %cd "/content/drive/MyDrive/stylegan3-fun-blend"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !git clone https://github.com/adamdavidcole/stylegan3-fun-blend.git
    %cd stylegan3-fun-blend
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    # !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/adamdavidcole/stylegan3-fun-blend.git
    %cd stylegan3-fun-blend
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Update all code files in drive repo 
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- "*.py" 
!git checkout origin/main -- "*.ipynb"

Already up to date.
Saved working directory and index state WIP on main: 59cff72 drive+collab test3


In [5]:
!pip install einops ninja gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 23.5 MB/s 


In [6]:
import numpy as np
from datetime import datetime

## Network Setup

In [8]:
network_key = "butterfly"
cfg="stylegan3-t"
pretrained_network = "ffhqu256"
dataset = "/content/drive/MyDrive/stylegan3/datasets/butterflys_256_2-256x256.zip"

results_outdir = f"./results/{network_key}"

if not os.path.isdir(results_outdir):
    !mkdir -p $results_outdir

## Training

In [ ]:
!python train.py --outdir=$outdir --cfg=$cfg --data=$dataset \
    --gpus=1 --batch=32 --batch-gpu=16  --gamma=6.6 --mirror=1 --kimg=50 --snap=1 --tick=1 --img-snap=1 --cbase=16384 \
    --resume=ffhqu256

## Projection

In [34]:
projection_source_images_outdir = "projection_source_images"
projection_source_vectors_outdir = "projection_source_vectors"


In [17]:
# Upload File 
if not os.path.isdir(projection_source_images_outdir):
  !mkdir -p $projection_source_images_outdir

def upload_files():
  filepaths = []
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    filepath = f"{projection_source_images_outdir}/{k}"
    open(filepath, 'wb').write(v)
    filepaths.append(filepath)
  return list(filepaths)

uploaded = upload_files();
print(uploaded)

Saving fakes_init_01.png to fakes_init_01.png
['projection_source_images/fakes_init_01.png']


In [ ]:
# Project Image
# uploaded_file = uploaded[0]
uploaded_file="/content/drive/MyDrive/stylegan3-fun-blend/results/butterfly/gen_images/seed0000.png"
uploaded_file_name = uploaded_file.split('/')[-1].split('.')[0:-1]
uploaded_file_name = ''.join(uploaded_file_name)
print(uploaded_file)

!python projector.py --outdir=$projection_source_vectors_outdir --target=$uploaded_file --project-in-wplus --num-steps=5000 --stabilize-projection \
       --cfg=stylegan3-r --network=ffhqu256

## Generation

In [9]:
most_recent_training_result = os.listdir(results_outdir)[-1]
print(most_recent_training_result)

path_to_most_recent_training_result = f"{results_outdir}/{most_recent_training_result}"
training_checkpoints = [f"{path_to_most_recent_training_result}/{f}" for f in os.listdir(path_to_most_recent_training_result) if f.endswith('.pkl')]
print(training_checkpoints)


00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256
['./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000000.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000001.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000002.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000003.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000004.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000005.pkl', './results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot

In [39]:
projected_w_path = f"{projection_source_vectors_outdir}/00001-projection-w-wavgstart-sgan2/projected_wavg_final.npy"
result_name="test_result"
# projection_network_pkl = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"];
# projection_network_pkl = 

projection_outdir=f"{results_outdir}/projections/{uploaded_file_name}"

print(projection_outdir)

if not os.path.isdir(projection_outdir):
  !mkdir -p $projection_outdir

./results/butterfly/projections/fakes_init_01


In [40]:
# Generate image from projection
projected_w = np.load(projected_w_path)
print(projected_w.shape)

(1, 16, 512)


In [ ]:
# Generate N images across network blend
import torch
import dnnlib
from dnnlib.util import format_time
import legacy
import PIL.Image

from torch_utils import gen_utils


# projection_network_pkl = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"]


def gen_img_from_network(network_pkl_path):
  network_pkl_name = network_pkl_path.split('/')[-1]
  print('Loading networks from "%s"...' % network_pkl_path)
  device = torch.device('cuda')
  with dnnlib.util.open_url(network_pkl_path) as fp:
      G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

  projected_w_tensor = torch.tensor(projected_w).to(device)
  synth_image = gen_utils.w_to_img(G, dlatents=projected_w_tensor, noise_mode='const')[0]
  PIL.Image.fromarray(synth_image, 'RGB').save(f'{projection_outdir}/{network_pkl_name}.jpg')

for training_checkpoint in training_checkpoints:
  gen_img_from_network(training_checkpoint)

## Debugging Image Generation

My images dont look similar to the seeds so I'm investigating if there is a discrepency in my generation pipeline

In [118]:
generated_images_outdir = f"{results_outdir}/gen_images"
# generated_images_network = gen_utils.resume_specs["stylegan3-r"]["ffhqu256"];
# network_name = "stylegan3-r_ffhqu256"

generated_images_network = training_checkpoints[13];
network_name = f"{network_key}_{generated_images_network.split('/')[-1]}"

In [ ]:
generated_images_version_count = len(os.listdir(generated_images_outdir))
generated_images_version_count_padded = f'{generated_images_version_count:04}'
generated_images_outdir_version = f"{generated_images_outdir}/{generated_images_version_count_padded}_{network_name}"
print(f"Outputting to {generated_images_outdir_version}")

!python gen_images.py --outdir=$generated_images_outdir_version --trunc=1 --seeds=0 --save_vectors=True \
        --network=$generated_images_network

In [106]:
from torch_utils import gen_utils

def z_to_img(G, latents: torch.Tensor, label: torch.Tensor, truncation_psi: float, noise_mode: str = 'const') -> np.ndarray:
    """
    Get an image/np.ndarray from a latent Z using G, the label, truncation_psi, and noise_mode. The shape
    of the output image/np.ndarray will be [len(latents), G.img_resolution, G.img_resolution, G.img_channels]
    """
    dlatents = gen_utils.z_to_dlatent(G=G, latents=latents, label=label, truncation_psi=truncation_psi)
    dlatents = G.mapping.w_avg + (G.mapping.w_avg - dlatents) * truncation_psi
    img = gen_utils.w_to_img(G=G, dlatents=dlatents, noise_mode=noise_mode)  # Let's not redo code
    return img


In [119]:
### Generate image from Z
from torch_utils import gen_utils

z_path = "/content/drive/MyDrive/stylegan3-fun-blend/results/butterfly/gen_images/0005_stylegan3-r_ffhqu256/0000_z.npy"

def gen_img_from_network_and_z(network_pkl_path, z_file_path):
  file_output_dir = f'{generated_images_outdir_version}/z_to_img2.jpg'

  network_pkl_name = network_pkl_path.split('/')[-1]
  print('Loading networks from "%s"...' % network_pkl_path)
  device = torch.device('cuda')
  with dnnlib.util.open_url(network_pkl_path) as fp:
      G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device)

  z_np = np.load(z_file_path)[0]
  z = torch.tensor(z_np).to(device)
  label = torch.zeros([1, G.c_dim], device=device)

  # These produce DIFFERENT results ?!?!?
  # synth_image = z_to_img(G, latents=z, label=label, truncation_psi=1)[0]
  # PIL.Image.fromarray(synth_image, 'RGB').save(file_output_dir)

  img = G(z, label, truncation_psi=1, noise_mode="const")
  img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
  PIL.Image.fromarray(img[0], 'RGB').save(file_output_dir)


  print(f"Outputted file to {file_output_dir}")
gen_img_from_network_and_z(generated_images_network, z_path)

Loading networks from "./results/butterfly/00005-stylegan3-r-butterflys_256_2-256x256-gpus1-batch32-gamma6.6-resume_ffhqu256/network-snapshot-000013.pkl"...
tensor([[ 1.7641,  0.4002,  0.9787,  2.2409,  1.8676, -0.9773,  0.9501, -0.1514,
         -0.1032,  0.4106,  0.1440,  1.4543,  0.7610,  0.1217,  0.4439,  0.3337,
          1.4941, -0.2052,  0.3131, -0.8541, -2.5530,  0.6536,  0.8644, -0.7422,
          2.2698, -1.4544,  0.0458, -0.1872,  1.5328,  1.4694,  0.1549,  0.3782,
         -0.8878, -1.9808, -0.3479,  0.1563,  1.2303,  1.2024, -0.3873, -0.3023,
         -1.0486, -1.4200, -1.7063,  1.9508, -0.5097, -0.4381, -1.2528,  0.7775,
         -1.6139, -0.2127, -0.8955,  0.3869, -0.5108, -1.1806, -0.0282,  0.4283,
          0.0665,  0.3025, -0.6343, -0.3627, -0.6725, -0.3596, -0.8131, -1.7263,
          0.1774, -0.4018, -1.6302,  0.4628, -0.9073,  0.0519,  0.7291,  0.1290,
          1.1394, -1.2348,  0.4023, -0.6848, -0.8708, -0.5788, -0.3116,  0.0562,
         -1.1651,  0.9008,  0.465

## Selection

## Refinement